In [9]:
import pandas as pd
import numpy as np
import ReciprocalRank
from tqdm import tqdm_notebook 

In [2]:
train = pd.read_csv('train.csv')

### example: user_id '008UN2RY0DMY'로 y값을 알아보자  

In [17]:
train.loc[train.user_id == '008UN2RY0DMY'][['reference','impressions']].iloc[22]

reference                                                3174122
impressions    63711|76170|3174122|3227059|561711|60144|60932...
Name: 14563099, dtype: object

In [14]:
rc = train.loc[train.user_id == '008UN2RY0DMY'][['reference','impressions']].iloc[22,1]

In [16]:
rc.count('|') + 1

25

* 이 user는 25개의 impressions(추천 list) 중 3번째에 ranking 된 item_id `3174122`를 구매했다. 

# train.csv EDA

In [28]:
train.tail()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
15932987,ZSJ7ZNLB6JQH,1d1948db73412,1541547574,155,interaction item image,2024423,BR,"Porto Seguro, Brazil",mobile,NaN,NaN,NaN
15932988,ZSJ7ZNLB6JQH,1d1948db73412,1541547574,156,interaction item info,2024423,BR,"Porto Seguro, Brazil",mobile,NaN,NaN,NaN
15932989,ZSJ7ZNLB6JQH,1d1948db73412,1541547597,157,interaction item image,2813114,BR,"Porto Seguro, Brazil",mobile,NaN,NaN,NaN
15932990,ZSJ7ZNLB6JQH,1d1948db73412,1541547597,158,interaction item image,2277390,BR,"Porto Seguro, Brazil",mobile,NaN,NaN,NaN
15932991,ZSJ7ZNLB6JQH,1d1948db73412,1541547744,159,clickout item,2277390,BR,"Porto Seguro, Brazil",mobile,NaN,2813114|1291318|11142246|477981|2784918|227739...,108|77|185|156|168|86|112|224|161|123|188|147|...


In [5]:
train.shape # 1600만개

(15932992, 12)

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15932992 entries, 0 to 15932991
Data columns (total 12 columns):
user_id            object
session_id         object
timestamp          int64
step               int64
action_type        object
reference          object
platform           object
city               object
device             object
current_filters    object
impressions        object
prices             object
dtypes: int64(2), object(10)
memory usage: 1.4+ GB


`user_id`

In [6]:
train.user_id.nunique() # user_id: 73만 user

730803

`session_id`

In [7]:
train.session_id.nunique()

910683

In [35]:
# user와 session의 관계 파악
# 각 user 마다 몇 번의 session이 나오는지 FE

`timestamp`

In [29]:
# timestamp(UNIX)를 local date, time으로 변환 
import datetime
def myfunc(x):
    return datetime.datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S')

mylist = list(map(myfunc,list(train.timestamp)))


In [32]:
train['time'] = mylist

In [33]:
train.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,time
0,008UN2RY0DMY,cd10940538005,1541031243,1,search for destination,"Rochester, USA",US,"Rochester, USA",desktop,NaN,NaN,NaN,2018-11-01 09:14:03
1,008UN2RY0DMY,cd10940538005,1541031463,2,change of sort order,interaction sort button,US,"Rochester, USA",desktop,NaN,NaN,NaN,2018-11-01 09:17:43
2,008UN2RY0DMY,cd10940538005,1541031473,3,change of sort order,distance and recommended,US,"Rochester, USA",desktop,NaN,NaN,NaN,2018-11-01 09:17:53
3,008UN2RY0DMY,cd10940538005,1541031473,4,filter selection,Focus on Distance,US,"Rochester, USA",desktop,Focus on Distance,NaN,NaN,2018-11-01 09:17:53
4,008UN2RY0DMY,cd10940538005,1541031550,5,interaction item image,57103,US,"Rochester, USA",desktop,NaN,NaN,NaN,2018-11-01 09:19:10


`step` = step in the sequence of actions within the session

`action_type`

In [37]:
train.action_type.value_counts()

interaction item image    11860750
clickout item              1586586
filter selection            695917
interaction item deals      411040
search for destination      403066
change of sort order        400584
interaction item info       285402
search for item             152203
search for poi              137444
Name: action_type, dtype: int64

In [38]:
# 각각을 컬럼으로 만들기 FE!!! 

`reference`= reference value of the action as described for the different action types

In [39]:
train.groupby('action_type')['reference'].value_counts()

action_type           reference                       
change of sort order  interaction sort button             235027
                      price only                           78863
                      price and recommended                43317
                      distance only                        12413
                      distance and recommended             10102
                      rating and recommended                9833
                      rating only                           7159
                      our recommendations                   3870
clickout item         8796                                   822
                      8561                                   763
                      8621                                   726
                      8589                                   652
                      1455251                                648
                      8586                                   634
                      8618         

In [ ]:
# action column의 세부 category FE !! 

`impression`: 상품 구입할 때 디스플레이된 상품 리스트 (추천시스템 적용 전)

# test.csv EDA

In [14]:
test = pd.read_csv('test.csv')

In [19]:
sum(test.loc[test.action_type == 'clickout item']['impressions'].isna())

0

* 제품 구매 시 impression 정보가 빠진 경우는 없다. 

In [13]:
test.loc[test.user_id == '000324D9BBUC']

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,000324D9BBUC,89643988fdbfb,1541593272,1,search for destination,"Budapest, Hungary",BG,"Budapest, Hungary",desktop,NaN,NaN,NaN
1,000324D9BBUC,89643988fdbfb,1541593820,2,filter selection,Car Park,BG,"Budapest, Hungary",desktop,Car Park,NaN,NaN
2,000324D9BBUC,89643988fdbfb,1541593882,3,interaction item image,1000915,BG,"Budapest, Hungary",desktop,NaN,NaN,NaN
3,000324D9BBUC,89643988fdbfb,1541593882,4,interaction item image,1000915,BG,"Budapest, Hungary",desktop,NaN,NaN,NaN
4,000324D9BBUC,89643988fdbfb,1541593902,5,interaction item image,1000915,BG,"Budapest, Hungary",desktop,NaN,NaN,NaN
5,000324D9BBUC,89643988fdbfb,1541593902,6,interaction item image,1000915,BG,"Budapest, Hungary",desktop,NaN,NaN,NaN
6,000324D9BBUC,89643988fdbfb,1541593931,7,interaction item image,1241375,BG,"Budapest, Hungary",desktop,NaN,NaN,NaN
7,000324D9BBUC,89643988fdbfb,1541593941,8,interaction item image,1241375,BG,"Budapest, Hungary",desktop,NaN,NaN,NaN
8,000324D9BBUC,89643988fdbfb,1541593941,9,interaction item image,1241375,BG,"Budapest, Hungary",desktop,NaN,NaN,NaN
9,000324D9BBUC,89643988fdbfb,1541593942,10,clickout item,NaN,BG,"Budapest, Hungary",desktop,NaN,923407|1729121|1050792|97171|353141|106315|218...,47|78|76|76|77|79|55|75|78|69|77|56|72|61|47|6...


* 구매한 상품 정보가 없다.

'action_type == clickout item' and 'reference == NaN' 

* 그럼 impression을 보고 어떤 itme_id를 샀는 지 맞춰?

In [20]:
sample = pd.read_csv('submission_popular.csv')

In [22]:
sample.loc[sample.user_id == '000324D9BBUC']

,user_id,session_id,timestamp,step,item_recommendations
0,000324D9BBUC,89643988fdbfb,1541593942,10,924795 106315 1033140 119494 101758 903037 105...


* item_recommendations는 결국 clickout_item을 맞추는 거네! 
* test에서 clickout 없는 session_id 개수와 sample의 row개수가 같은가?

In [23]:
sample.shape

(253573, 5)

In [25]:
sum(test.loc[test.action_type == 'clickout item']['reference'].isna())

253573

* yes! 

# Conclusion
`submission`의 `item_recommendations`를 만드는 것이 대회의 최종 목표이다. test의 session_id중에서 유저가 산 item_id가 빠져있는 (nan) 데이터가 있다. (253,572개) `item_recommendations`는 이 유저들이 가장 살 만한 아이템의 추천 리스트라고 볼 수 있다. 